# 自动求导

假设我们想对函数𝑦=2𝐱^⊤𝐱关于列向量𝐱求导

In [1]:
import torch

x = torch.arange(4.0) # 生成一个包含 [0, 1, 2, 3] 的 1D 张量
x

tensor([0., 1., 2., 3.])

在我们计算𝑦关于𝐱的梯度之前，我们需要一个地方来存储梯度

In [2]:
x.requires_grad_(True) # 开启 x 的梯度计算功能，使其能够记录计算图。
x.grad # 此时还未计算梯度，所以 x.grad 为空

现在让我们计算𝑦

In [3]:
y = 2 * torch.dot(x, x) 
y

tensor(28., grad_fn=<MulBackward0>)

以上输出中：
* grad_fn 是张量的梯度函数，用于记录生成这个张量的操作历史，以便在调用 backward() 时自动计算梯度。
* MulBackward0 表示这个张量是由一个乘法操作（Mul）生成的，这个操作对应于反向传播中的梯度计算逻辑。

通过调用反向传播函数来自动计算y关于x每个分量的梯度

In [4]:
y.backward() # 触发反向传播，计算 y 关于 x 的梯度
x.grad

tensor([ 0.,  4.,  8., 12.])

In [5]:
x.grad == 4 * x # 验证梯度的正确性

tensor([True, True, True, True])

现在让我们计算x的另一个函数

In [6]:
x.grad.zero_() # 清空之前计算的梯度，避免累积影响
y = x.sum()
y.backward()
x.grad

tensor([1., 1., 1., 1.])

如果不进行清空，梯度会进行累加，如下：

In [7]:
x.grad.zero_() # 清空之前计算的梯度，避免累积影响
y = x.sum()
y.backward()
x.grad

# x.grad.zero_() # 清空之前计算的梯度，避免累积影响
y = x.sum()
y.backward()
x.grad

tensor([2., 2., 2., 2.])

深度学习中，我们的目的不是计算微分矩阵，而是批量中每个样本单独计算的偏导数之和

In [8]:
x.grad.zero_()
y = x * x
y.sum().backward()
x.grad

tensor([0., 2., 4., 6.])

将某些计算移动到记录的计算图之外

In [9]:
x.grad.zero_()
y = x * x
u = y.detach() # 将 y 的计算从计算图中分离，u 不会记录梯度。
z = u * x

z.sum().backward()
x.grad == u

tensor([True, True, True, True])

In [10]:
x.grad.zero_()
y.sum().backward()
x.grad == 2 * x

tensor([True, True, True, True])

即使构建函数的计算图需要通过Python控制流（例如，条件、循环或任意函数调用），我们仍然可以计算得到的变量的梯度

In [11]:
def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

a = torch.randn(size=(), requires_grad=True)
d = f(a)
d.backward()

a.grad == d / a

tensor(True)

* PyTorch 的自动求导依赖于动态计算图，能够处理任意复杂的函数。
* 梯度计算结果可以通过 .backward() 自动完成，无需显式推导。
* 对于深度学习中的大多数任务，自动求导极大地简化了模型优化和梯度计算的流程。

In [12]:
import torch

# 定义矩阵
X = torch.randn(5, 3, requires_grad=True)  # 输入
W = torch.randn(4, 5, requires_grad=True)  # 权重
b = torch.randn(4, 1, requires_grad=True)  # 偏置

# 正向传播
Z = W @ X + b
A = torch.sigmoid(Z)  # 激活函数
loss = A.mean()  # 假设损失函数为平均值

# 反向传播
loss.backward()

# 查看梯度
print("dL/dW:", W.grad)
print("dL/db:", b.grad)

dL/dW: tensor([[ 0.0086,  0.0050, -0.0010, -0.0095, -0.0211],
        [ 0.0333,  0.0063,  0.0209, -0.0097,  0.0101],
        [ 0.0557,  0.0181,  0.0228, -0.0304, -0.0309],
        [ 0.0337,  0.0092,  0.0160, -0.0155, -0.0091]])
dL/db: tensor([[0.0111],
        [0.0329],
        [0.0607],
        [0.0354]])
